# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/vijay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vijay/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

mkdir: data: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31528    0 31528    0     0  57539      0 --:--:-- --:--:-- --:--:-- 57532


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70695    0 70695    0     0   483k      0 --:--:-- --:--:-- --:--:--  486k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/vijay/ai-engineering-bootcamp/aie-bootcamp/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/vijay/ai-engineering-bootcamp/aie-bootcamp/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/vijay/ai-engineering-bootcamp/aie-bootcamp/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 11, relationships: 31)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 11, relationships: 31)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


RAGAS provides **three types of query synthesizers**, each simulating a different style of question that a user might ask. Here's what they do in simple terms:

1. **`SingleHopSpecificQuerySynthesizer`**

   * **Purpose**: Generates **simple, fact-based questions** that can be answered using a single passage or fact.
   * **Example**: “Who created the Llama model?”
   * **Use Case**: Mimics straightforward user queries, typically found in FAQs or product info.

2. **`MultiHopAbstractQuerySynthesizer`**

   * **Purpose**: Generates **more abstract or reasoning-heavy questions** that span **multiple facts or ideas**.
   * **Example**: “How does OpenAI’s voice mode impact the ethical use of AI?”
   * **Use Case**: Simulates users who are **analyzing patterns or drawing conclusions**, common in expert reviews or research.

3. **`MultiHopSpecificQuerySynthesizer`**

   * **Purpose**: Generates **multi-hop questions** that are **fact-based but require combining multiple sources** to answer.
   * **Example**: “What did Meta and Google each contribute to AI in 2024?”
   * **Use Case**: Mimics advanced users doing comparative or investigative queries.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Is Google a company that develops LLMs?,[My blog in 2023 Here’s the sequel to this pos...,"Yes, the context mentions that Google is one o...",single_hop_specifc_query_synthesizer
1,how does Python help in making AI models like ...,[openly licensed ones are still the world’s mo...,"The context mentions that writing code, includ...",single_hop_specifc_query_synthesizer
2,What are Large Language Models according to th...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Large Language Models are the latest and most ...,single_hop_specifc_query_synthesizer
3,How does ChatGPT relate to the broader landsca...,"[of what LLMs are, how they work and how they ...",The context discusses ChatGPT as part of the l...,single_hop_specifc_query_synthesizer
4,What is the Chatbot Arena and how has it chang...,[Everything tagged “llms” on my blog in 2024 T...,The Chatbot Arena is a leaderboard that ranks ...,single_hop_specifc_query_synthesizer
5,How do recent advancements in AI regulation an...,[<1-hop>\n\nEverything tagged “llms” on my blo...,"Recent advancements in AI regulation, such as ...",multi_hop_abstract_query_synthesizer
6,How do the legal and ethical considerations su...,[<1-hop>\n\nopenly licensed ones are still the...,The development and deployment of large langua...,multi_hop_abstract_query_synthesizer
7,Whi is the cost of trainin a large langwage mo...,[<1-hop>\n\na relevant paper Training Large La...,"DeepSeek v3 was trained on 2,788,000 GPU hours...",multi_hop_abstract_query_synthesizer
8,how does Claude 3 compare to Claude in the con...,[<1-hop>\n\na relevant paper Training Large La...,The context mentions Claude 3 and Claude as th...,multi_hop_specific_query_synthesizer
9,Whos Simon Willison and how does he talk about...,"[<1-hop>\n\nof what LLMs are, how they work an...",Simon Willison is a blogger who writes extensi...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/22 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is xAI in context of LLMs?,[My blog in 2023 Here’s the sequel to this pos...,xAI is mentioned as one of the organizations t...,single_hop_specifc_query_synthesizer
1,What are the key challenges and developments r...,[openly licensed ones are still the world’s mo...,"In 2024, openly licensed LLMs remain complex b...",single_hop_specifc_query_synthesizer
2,Whaat are the main advancments and chalenges w...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"According to Simon Willison’s weblog, 2023 was...",single_hop_specifc_query_synthesizer
3,What is Stanford Alpaca and how does it relate...,"[of what LLMs are, how they work and how they ...",Stanford Alpaca is mentioned in the context of...,single_hop_specifc_query_synthesizer
4,How does the recent development and increased ...,"[<1-hop>\n\nof what LLMs are, how they work an...",The recent development and widespread adoption...,multi_hop_abstract_query_synthesizer
5,How do the performance and compatibility of AI...,[<1-hop>\n\nAny systems that attempts to make ...,The performance and compatibility of AI models...,multi_hop_abstract_query_synthesizer
6,How do advancements in AI hardware and infrast...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s 2024 weblog highlights signif...,multi_hop_abstract_query_synthesizer
7,how prompt injection and gullibility make AI s...,[<1-hop>\n\nopenly licensed ones are still the...,prompt injection and gullibility are big probl...,multi_hop_abstract_query_synthesizer
8,how Llama 2 and Llama 3.3 are related and why ...,[<1-hop>\n\nThe environmental impact got bette...,"In 2024, Llama 2 and Llama 3.3 are significant...",multi_hop_specific_query_synthesizer
9,Claude and Gemini both can do what with AI?,"[<1-hop>\n\nalso accepts audio input, and the ...",Claude can write on-demand interactive applica...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/vijay/ai-engineering-bootcamp/aie-bootcamp/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/vijay/ai-engineering-bootcamp/aie-bootcamp/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term that lacks a single, clear, and widely understood meaning. Generally, the term seems to be converging on AI systems that can go away and act on your behalf. There are two main ways people think of AI agents: one is the "travel agent" model, where the agent acts on your behalf, and the other involves large language models (LLMs) that have access to tools and can run them in loops to solve problems. However, despite excitement and many prototypes, fully functional AI agents running in production are scarce, partly due to challenges like gullibility—LLMs tend to believe any input, which limits their ability to make meaningful decisions autonomously. In sum, "agents" are AI systems intended to act autonomously for users, but the concept remains nebulous and largely unrealized at this time.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

**`qa_evaluator`**

* **Evaluates factual correctness**
* Compares the model's output (`prediction`) with the provided ground-truth (`reference`)
* Ensures the answer is **accurate and grounded in context**

---

**`labeled_helpfulness_evaluator`**

* **Evaluates helpfulness**, **given a known reference answer**
* Checks if the answer is **helpful to the user**, not just correct
* Takes into account **clarity**, **relevance**, and **completeness**
* Uses the reference answer to determine how well the prediction assists the user

---

**`dope_or_nope_evaluator`**

* Evaluates the **“coolness” or “dopeness”** of the output
* Assesses if the answer is **engaging, impressive, or human-like**
* Captures qualitative fluency and user satisfaction aspects that go beyond correctness

---

These three evaluators together provide a **multi-faceted view** of LLM output quality:

* `qa` → factual accuracy
* `helpfulness` → practical utility
* `dopeness` → engagement and tone

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'proper-potato-60' at:
https://smith.langchain.com/o/cc1700a3-14e9-4dd6-9611-c9c0b0000632/datasets/4e9298ed-3f62-4bbc-bdbc-5df4336101b1/compare?selectedSessions=0a247c3a-65e9-4e35-a1dc-7da1e201ad7d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the advancements in Llama 3.2 and Llama...,"Based on the provided context, advancements in...",None,The context highlights that models like Llama ...,1,1,0,5.964481,8178e799-bff3-4a6c-ba6c-d999aeab19f5,35763cf9-7091-4301-bc56-aa9dfb90509a
1,How does the development of models like Claude...,"Based on the provided context, models like Cla...",None,The context highlights that models like Claude...,1,1,0,5.995312,f9bf3eaf-03a6-42e9-a763-cff540cdabe5,9000723f-8e38-4c99-820f-93a9b7e685d1
2,Claude and Gemini both can do what with AI?,"Based on the context, both Claude and Gemini c...",None,Claude can write on-demand interactive applica...,0,0,0,1.857599,7e37c379-e4fa-4b5d-aa0d-20bebe77b4cd,b5acd391-0f20-4181-9538-fa6c7a834c3f
3,how Llama 2 and Llama 3.3 are related and why ...,I don't know.,None,"In 2024, Llama 2 and Llama 3.3 are significant...",0,0,0,1.430459,47b59bae-f356-4e15-a34a-bd272768b090,dfdc3371-ecd8-4f98-88a0-df2d543a0fbc
4,how prompt injection and gullibility make AI s...,Based on the provided context:\n\nPrompt injec...,None,prompt injection and gullibility are big probl...,1,1,0,4.204320,77304db6-8ca8-4de6-a0f3-89400c2160e8,e3b3ff05-dbbf-49c3-bd57-721ae1d54a18
5,How do advancements in AI hardware and infrast...,I don't know.,None,Simon Willison’s 2024 weblog highlights signif...,0,0,0,1.305058,6d9ab1ba-dcd7-49e8-8284-616502bd8940,c4f81bde-6ecf-4694-a289-1e4040668517
6,How do the performance and compatibility of AI...,"Based on the context, the performance and comp...",None,The performance and compatibility of AI models...,1,1,0,4.825255,12a365bc-3d4c-41c0-bd8d-44f96ca451ba,f3e1168e-de26-4ab8-8fcc-911cff9487c6
7,How does the recent development and increased ...,"Based on the provided context, the recent deve...",None,The recent development and widespread adoption...,1,1,0,4.495899,76eb452c-76ca-49e4-b436-4b31071bd0e9,efa423a2-4f95-4ea1-97c3-2cda9b880408
8,What is Stanford Alpaca and how does it relate...,"Based on the provided context, Stanford Alpaca...",None,Stanford Alpaca is mentioned in the context of...,1,1,0,3.368660,ffc3b831-4a98-4a52-996f-00980eb17e4c,e527b10b-60fb-41ab-a08c-48b6467dc5ab
9,Whaat are the main advancments and chalenges w...,"Based on the provided context, the main advanc...",None,"According to Simon Willison’s weblog, 2023 was...",1,1,0,5.038086,de7f6dda-e1e7-48bf-9cc6-6dcb59295276,049f8154-f36e-47d8-8c80-5ef15a154be5


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [5]:
rag_documents = docs

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### Question #2:

Why would modifying our chunk size modify the performance of our application?

Modifying the **chunk size** in a RAG pipeline directly impacts the **retriever's ability to retrieve relevant and complete context** for a given query. Here’s why:

**1. Larger Chunks Preserve More Context**

* Larger chunks (e.g. 1000 tokens vs. 500) retain **more semantic coherence** from the original document.
* This reduces the chance that important details are split across chunks.

**2. Improved Retrieval Quality**

* With more context per chunk, the retriever can fetch **more meaningful and self-contained passages**, improving grounding.
* The LLM gets better input, leading to more accurate and complete answers.

**3. Tradeoff: Chunk Size vs. Recall Granularity**

* Smaller chunks give **finer-grained control**, but may miss key context due to fragmentation.
* Larger chunks increase **contextual richness**, but may include more noise.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### Question #3:

Why would modifying our embedding model modify the performance of our application?

Changing the **embedding model** directly affects the **retriever's quality** in a Retrieval-Augmented Generation (RAG) system. Here's why:

---

**1. Higher-Quality Embeddings = Better Similarity Search**

* A more advanced embedding model (like `text-embedding-3-large` vs. `text-embedding-3-small`) creates **more semantically rich and accurate vector representations** of text.
* This improves the retriever's ability to fetch **relevant and contextually precise chunks**, even for abstract or multi-hop queries.

---

**2. More Nuanced Understanding of Queries**

* Larger models embed **deeper linguistic and conceptual information**, improving their ability to match nuanced questions with appropriate chunks.
* Especially important when documents use **varied or indirect phrasing**.

---

**3. Direct Impact on Downstream LLM Output**

* If the retriever sends better chunks, the LLM has a **richer and more complete context**, leading to:

  * More accurate answers
  * Better grounding
  * Less hallucination

---

**Tradeoff:**

* Larger embedding models can be **slightly more expensive and slower**, but the gain in **retrieval relevance and final output quality** often outweighs this.

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown on "Agents" straight from the context — it’s a term wrapped in mystery and hype but lacking a solid, agreed-upon definition. Some folks think of agents like travel agents — AI doing stuff on your behalf, handling your errands digitally. Others see agents as LLMs (large language models) hooked up with tools, running loops to solve problems autonomously.\n\nBut here’s the kicker: agents haven’t really arrived in full force yet. They\'re often stuck in the “coming soon” zone, mainly because of that pesky problem called gullibility — these models buy into anything you feed them, even nonsense or fake info, which makes fully autonomous decisions sketchy.\n\nSo bottom line? "Agents" are basically AI systems that are supposed to act independently for you, but they’re still a work-in-progress, more sci-fi than everyday reality. Trying to build them without solving core issues like trustworthiness and truth detection is like trying to build a spaceship with duct tap

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'clear-jump-66' at:
https://smith.langchain.com/o/cc1700a3-14e9-4dd6-9611-c9c0b0000632/datasets/4e9298ed-3f62-4bbc-bdbc-5df4336101b1/compare?selectedSessions=c7fc3082-f6bb-438b-9c6b-de7d27ad7c5a




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the advancements in Llama 3.2 and Llama...,"Alright, here’s the lowdown, straight from the...",None,The context highlights that models like Llama ...,1,1,1,4.166944,8178e799-bff3-4a6c-ba6c-d999aeab19f5,a9df8fab-32e5-4115-932c-e2d26a7e5cdb
1,How does the development of models like Claude...,"Alright, here’s the lowdown, straight fire sty...",None,The context highlights that models like Claude...,1,1,1,3.364831,f9bf3eaf-03a6-42e9-a763-cff540cdabe5,ec6b59d3-84ae-48bc-8b6e-d151d81dba00
2,Claude and Gemini both can do what with AI?,"Yo, check it—both Claude and Gemini are flexin...",None,Claude can write on-demand interactive applica...,0,0,1,3.856451,7e37c379-e4fa-4b5d-aa0d-20bebe77b4cd,2056e970-90e4-44a3-9044-bc2259af58ca
3,how Llama 2 and Llama 3.3 are related and why ...,"Yo, here’s the deal — the context’s got the sp...",None,"In 2024, Llama 2 and Llama 3.3 are significant...",0,0,1,4.449739,47b59bae-f356-4e15-a34a-bd272768b090,ace3b32d-b816-4b4c-ac86-a8d73618b842
4,how prompt injection and gullibility make AI s...,"Alright, listen up! Prompt injection and gulli...",None,prompt injection and gullibility are big probl...,1,1,1,3.370971,77304db6-8ca8-4de6-a0f3-89400c2160e8,3ffcbaa5-a702-447e-bf8c-99b92a782256
5,How do advancements in AI hardware and infrast...,"Yo, check it out — Simon Willison’s 2024 vibe ...",None,Simon Willison’s 2024 weblog highlights signif...,1,1,1,6.240872,6d9ab1ba-dcd7-49e8-8284-616502bd8940,25ffb43f-9149-4bd4-acec-cb4ed397f1ee
6,How do the performance and compatibility of AI...,"Alright, here’s the lowdown, fresh and clean:\...",None,The performance and compatibility of AI models...,1,1,1,4.562091,12a365bc-3d4c-41c0-bd8d-44f96ca451ba,cedc01c8-5869-4f5c-a533-30e1a61c0391
7,How does the recent development and increased ...,"Alright, let’s unpack this AI storytelling vib...",None,The recent development and widespread adoption...,1,1,1,5.733108,76eb452c-76ca-49e4-b436-4b31071bd0e9,78b1e5bf-f62e-4385-912b-fb13a173af63
8,What is Stanford Alpaca and how does it relate...,"Alright, here’s the scoop straight from the co...",None,Stanford Alpaca is mentioned in the context of...,1,1,1,2.324113,ffc3b831-4a98-4a52-996f-00980eb17e4c,59887377-3ba5-464f-9cce-e9c6a642f926
9,Whaat are the main advancments and chalenges w...,"Yo, let’s break down the 2023 LLM scene like a...",None,"According to Simon Willison’s weblog, 2023 was...",1,1,1,3.022762,de7f6dda-e1e7-48bf-9cc6-6dcb59295276,08d713a1-5c85-4803-9b3c-dd03d7b88b7e


#### Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![First Chain](./images/chain-1.png)

![Second Chain](./images/chain-2.png)

#### **Chain Evaluation Comparison – LangSmith Analysis**

This activity compares two RAG chains evaluated using LangSmith: a baseline chain (`proper-potato-60`) and an enhanced, stylistically optimized chain (`clear-jump-66`). The goal is to analyze how modifications in the chain design impacted evaluation metrics such as correctness, helpfulness, and dopeness.

---

##### **Screenshot Comparison**

| Chain Name         | Screenshot | Style        | Prompt Type         | Embedding Model           | Chunk Size | Notable Features           |
|--------------------|------------|---------------|----------------------|----------------------------|------------|------------------------------|
| `proper-potato-60` | ![Chain 1](./images/chain-1.png) | Factual & Formal | Baseline Prompt      | `text-embedding-3-small` | 500        | Straightforward factual answers |
| `clear-jump-66`    | ![Chain 2](./images/chain-2.png) | Dope & Engaging | Stylized “cool” prompt | `text-embedding-3-large` | 1000       | Tone-tuned, engaging responses   |


---

##### **Evaluation Metric Changes**

| Metric        | Baseline (`proper-potato-60`) | Dope Chain (`clear-jump-66`) | Change Summary |
|---------------|-------------------------------|-------------------------------|----------------|
| **Correctness**   | 0.75 avg                          | 0.75 avg                          | Unchanged – Strong grounding in both |
| **Helpfulness**   | 0.75 avg                          | 0.67 avg                          | Slight drop – Some responses prioritized tone over clarity |
| **Dopeness**      | **0.00 avg**                     | **1.00 avg**                     | Significant gain – Style optimized responses were more engaging |
| **Latency (P50)** | ~4.35s                            | ~4.01s                            | Minor improvement – Likely due to faster model |
| **Tokens Used**   | 22,327                            | 16,470                            | Fewer tokens – More concise, stylistic output |


---

##### **Observations & Insights**

<u>Dopeness ↑</u>
- Use of a tone-enhanced prompt (`"You must answer in a dope way"`) significantly improved dopeness scores.
- Model generated responses that felt more conversational and expressive (e.g., “Yo, here’s the deal…”).

<u>Helpfulness ↓</u>
- While still grounded, some responses sacrificed technical completeness for style.
- Informal tone may reduce perceived usefulness in academic or professional contexts.

<u>Correctness Stable</u>
- Larger chunk size and better embeddings preserved or improved retrieval quality.
- High correctness maintained across both chains.

<u>Tokens ↓</u>
- Despite larger chunks, the stylized outputs were more concise.
- Potential efficiency gain in generation cost.


---

##### Conclusion

The enhanced "Dope Chain" succeeded in improving **user engagement** and **language style** without compromising correctness. However, it introduced a small tradeoff in **helpfulness**, highlighting the need to carefully balance **tone vs clarity** in prompt engineering.

> This experiment demonstrates the power of iterative tuning using LangSmith and synthetic evaluations. Style-aware prompting, combined with better retrieval context and embeddings, can meaningfully affect both qualitative and quantitative performance metrics.



# **Bonus Activity**

In [8]:
# Synthetic Data Generation using LangGraph Agent Graph with Evol Instruct (Real Tools)

from typing import List, Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI

# -----------------------------
# Step 1: Define State Schema
# -----------------------------
class EvoState(TypedDict):
    documents: List[Document]
    evolved_questions: List[Dict]
    answers: List[Dict]
    contexts: List[Dict]

# -----------------------------
# Step 2: Real Tools
# -----------------------------
# Chat model for all stages
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Mock Evol Instruct Tool
class EvolInstructTool:
    def __call__(self, docs: List[Document]) -> List[Dict]:
        # Simulate 3 types of evolutions: simple, multi-context, reasoning
        return [
            {"question_id": "q1", "evolved_question": "What is GPT-4 used for?", "type": "simple"},
            {"question_id": "q2", "evolved_question": "Compare GPT-4 with Claude 3", "type": "multi-context"},
            {"question_id": "q3", "evolved_question": "Why is multimodal input critical in LLMs?", "type": "reasoning"}
        ]

# Answer tool
class AnswerTool:
    def __call__(self, questions: List[Dict]) -> List[Dict]:
        return [{"question_id": q["question_id"], "answer": llm.invoke(q["evolved_question"]).content} for q in questions]

# Context retriever tool
class ContextTool:
    def __call__(self, questions: List[Dict], docs: List[Document]) -> List[Dict]:
        # Naive context matcher for example
        return [{"question_id": q["question_id"], "contexts": [doc.page_content for doc in docs[:1]]} for q in questions]

# Tools
evol_tool = EvolInstructTool()
answer_tool = AnswerTool()
context_tool = ContextTool()

# -----------------------------
# Step 3: Nodes
# -----------------------------
def evol_node(state: EvoState) -> dict:
    docs = state['documents']
    questions = evol_tool(docs)
    return {"evolved_questions": questions}

def answer_node(state: EvoState) -> dict:
    return {"answers": answer_tool(state['evolved_questions'])}

def context_node(state: EvoState) -> dict:
    return {"contexts": context_tool(state['evolved_questions'], state['documents'])}

# -----------------------------
# Step 4: LangGraph Pipeline
# -----------------------------
builder = StateGraph(EvoState)
builder.add_node("evolve", evol_node)
builder.add_node("answer", answer_node)
builder.add_node("context", context_node)

builder.set_entry_point("evolve")
builder.add_edge("evolve", "answer")
builder.add_edge("evolve", "context")
builder.add_edge("answer", END)
builder.add_edge("context", END)

runnable = builder.compile()

# -----------------------------
# Step 5: Sample Run
# -----------------------------
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()
final_state = runnable.invoke({"documents": docs})

# -----------------------------
# Step 6: Output
# -----------------------------
from pprint import pprint
print("\nEvolved Questions:")
pprint(final_state['evolved_questions'])

print("\nAnswers:")
pprint(final_state['answers'])

print("\nContexts:")
pprint(final_state['contexts'])

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.



Evolved Questions:
[{'evolved_question': 'What is GPT-4 used for?',
  'question_id': 'q1',
  'type': 'simple'},
 {'evolved_question': 'Compare GPT-4 with Claude 3',
  'question_id': 'q2',
  'type': 'multi-context'},
 {'evolved_question': 'Why is multimodal input critical in LLMs?',
  'question_id': 'q3',
  'type': 'reasoning'}]

Answers:
[{'answer': 'As of my knowledge up to October 2021, GPT-4 has not been '
            'released yet. However, it is expected that GPT-4, like its '
            'predecessor GPT-3, would be used for tasks that involve natural '
            'language understanding and generation. This could include tasks '
            'like translation, question answering, summarization, content '
            "creation, and more. It's also likely that it would be used in "
            'chatbots, virtual assistants, and other applications that require '
            'sophisticated language processing capabilities.',
  'question_id': 'q1'},
 {'answer': "As an AI developed b